In [1]:
import os


if os.getcwd().split("/")[-1] != "mlkube":
    os.chdir("..")
    os.getcwd()


import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for each image type
input_shape = (150, 150, 3)  # Adjust based on your images

# Define the path to your dataset folder
data_dir = 'data_source/Data'

# Define the classes and corresponding folder names
classes = [ "Normal","glioma_tumor","meningioma_tumor","pituitory_tumor" ]
folders = ['BR', 'DA', 'HF', 'n','RO','SP','VF']

# Function to load and preprocess the images for a given class and image type
def load_images(class_name, folder_name):
    folder_path = os.path.join(data_dir, class_name, folder_name)
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        print(img_path)
        raw = tf.io.read_file(img_path)
        img = tf.image.decode_image(raw)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
        images.append(img_array)
    return np.array(images)

# Load the images and create the inputs for each class and image type
inputs = []
for class_name in classes:
    class_inputs = []
    for folder_name in folders:
        images = load_images(class_name, folder_name)
        class_inputs.append(images)
    inputs.append(class_inputs)



2023-08-01 16:17:56.574478: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-01 16:17:57.714302: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-01 16:17:57.720636: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 16:18:00.818878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


data_source/Data/Normal/BR/N_100_BR_.jpg
data_source/Data/Normal/BR/N_101_BR_.jpg
data_source/Data/Normal/BR/N_102_BR_.jpg
data_source/Data/Normal/BR/N_103_BR_.jpg
data_source/Data/Normal/BR/N_104_BR_.jpg
data_source/Data/Normal/BR/N_105_BR_.jpg
data_source/Data/Normal/BR/N_106_BR_.jpg
data_source/Data/Normal/BR/N_107_BR_.jpg
data_source/Data/Normal/BR/N_108_BR_.jpg
data_source/Data/Normal/BR/N_109_BR_.jpg
data_source/Data/Normal/BR/N_10_BR_.jpg
data_source/Data/Normal/BR/N_110_BR_.jpg
data_source/Data/Normal/BR/N_111_BR_.jpg
data_source/Data/Normal/BR/N_112_BR_.jpg
data_source/Data/Normal/BR/N_113_BR_.jpg
data_source/Data/Normal/BR/N_114_BR_.jpg
data_source/Data/Normal/BR/N_115_BR_.jpg
data_source/Data/Normal/BR/N_116_BR_.jpg
data_source/Data/Normal/BR/N_117_BR_.jpg
data_source/Data/Normal/BR/N_118_BR_.jpg
data_source/Data/Normal/BR/N_119_BR_.jpg
data_source/Data/Normal/BR/N_11_BR_.jpg
data_source/Data/Normal/BR/N_120_BR_.jpg
data_source/Data/Normal/BR/N_121_BR_.jpg
data_source/Data/N

: 

: 

In [ ]:
# Define the model architecture using the functional API
def build_model(input_shape):
    inputs = [Input(shape=input_shape) for _ in range(4)]
    x = []
    for i in range(4):
        x.append(Conv2D(32, (3, 3), activation='relu')(inputs[i]))
        x[i] = MaxPooling2D(pool_size=(2, 2))(x[i])
        x[i] = Flatten()(x[i])
    x = tf.keras.layers.concatenate(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(len(classes), activation='softmax')(x)
    model = Model(inputs=inputs, outputs=output)
    return model

# Build the model
model = build_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the labels for each class (0 for normal, 1 for abnormal_TYPE1, 2 for abnormal_TYPE2)
labels = [0, 1, 2]

# Prepare the labels and data for training and testing
X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []

for i, class_inputs in enumerate(inputs):
    for img_type_input in class_inputs:
        num_samples = len(img_type_input)
        num_train = int(0.7 * num_samples)  # 70% for training
        num_val = int(0.15 * num_samples)  # 15% for validation
        num_test = num_samples - num_train - num_val  # Remaining for testing
        
        # Shuffle the data
        np.random.shuffle(img_type_input)

        # Split the data into training, validation, and test sets
        X_train.extend(img_type_input[:num_train])
        X_val.extend(img_type_input[num_train:num_train+num_val])
        X_test.extend(img_type_input[num_train+num_val:])
        y_train.extend([labels[i]] * num_train)
        y_val.extend([labels[i]] * num_val)
        y_test.extend([labels[i]] * num_test)

# Convert the data to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)
y_test = tf.keras.utils.to_categorical(y_test)

# Train the model
model.fit([X_train for _ in range(4)], y_train, validation_data=([X_val for _ in range(4)], y_val), epochs=10, batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate([X_test for _ in range(4)], y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")